<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from tensorflow import keras
import json
import spacy
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras .optimizers import SGD
import random

print(tf.__version__)

1.15.2


In [31]:
# loading json  data into a file  and converting it into dictionary  
data_file=open("intents.json").read()
intents=json.loads(data_file)

In [32]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [33]:
type(intents)

dict

In [34]:
nlp=spacy.load("en_core_web_sm")
words =[]
classes=[]
documents=[]
ignore_words=['?','!']


In [ ]:
#using shantanus approach
words=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern=str(pattern)
        w=[]
        #tokenize each word
        for word in pattern.split():
          if word != '.':
             word = word.replace('.','')
             w.append(word)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
print(w)
print(words)
print(len(words))
print(documents)
print(classes)

In [ ]:
text="hi my name is Manish"
tokens=nlp(text)
l=[]
for i in tokens:
    i=str(i)
    l.append(i)
print(l)

['hi', 'my', 'name', 'is', 'Manish']


In [ ]:
type(tokens)

spacy.tokens.doc.Doc

In [36]:
words=[]
documents=[]
classes=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        tokens= nlp(pattern)
        l=[]
        for i in tokens:
             i=str(i)
             l.append(i)
        words.extend(l)
        #add documents in the corpus
        documents.append((l, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [37]:
print(l)
print(words)
print(len(words))
print(documents)
print(classes)
len(classes)



['Looking', 'up', 'hospital', 'details']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'do', 'nt', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'e

9

In [78]:
#create our training data
training= []
# create an empty array for our output
output_empty=[0]*len(classes)
#training set, bag of words for each sentence
for doc in documents:
   # intialize our bag of words
   bag=[]
   #list of tokenized words fo pattern
   pattern_words=doc[0]
   #lemmatize each word -create base word, in attempt to represent related word
   pattern_words=[nlp(word.lower())[0].lemma_ for word in pattern_words]
    
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
  
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
trainx = list(training[:,0])
trainy = list(training[:,1])
print("Training data created")





Training data created


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [128]:
new_words=[]
for i in words:
    new_words.append(i.lower())
print(words)
print(new_words)

unique_words = set(words)
unique_words
word2int = {}
int2word = {}
vocab_size = len(unique_words)
print(vocab_size)
for i,word in enumerate(unique_words):
  word2int[word] = i
  int2word[i] = word 
print(word2int)
print(int2word)
sentences=list()
for i in documents:
    sentences.append(i[0])
print(sentences)    

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'do', 'nt', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood',

In [129]:
window_size = 2
data = []
for sentence in sentences :
  for wordId,word in enumerate(sentence):
    for nxword in sentence[max(wordId - window_size,0):min(wordId + window_size, len(sentence))+1]:
        if nxword != word:
           data.append([word,nxword])

data
len(data)

510

In [130]:
def Encoder(datapointindex, vocab_size):
  temp = np.zeros(vocab_size)
  temp[datapointindex] = 1
  return temp

In [131]:
train_x = []
train_y = []

for worddata in data:
  try:
    train_x.append(Encoder(word2int[worddata[0]], vocab_size))
    train_y.append(Encoder(word2int[worddata[1]], vocab_size))
  except:

    continue

In [132]:
len(train_y)


510

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(trainx[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(trainy[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(trainx), np.array(trainy), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


print("model created")

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model_2.h5', hist)


print("model created")